**Plan: deseq2 does not accept rows with identical names
Therefore, add a barcode to the intron location


To generate a list of introns, a script was used from: http://seqanswers.com/forums/showthread.php?t=42420

This script was used on the GTF.88 from ensembl

The generated file was further parsed below:

In [1]:
from __future__ import division 
import pandas as pd 
import csv 

In [2]:
# read in introns and exons file 

df = pd.read_table('introns_and_exons_88.gff', header=None, 
quoting=csv.QUOTE_NONE,
names = ['Chr','notes','part','Start','End','dot1','Strand','dot2','name'])


/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


In [3]:
#  Get onley the intron locations

df_intron = df[df.part.str.contains("intronic_part")]

df_intron.reset_index(inplace=True,drop=True)

df_intron = df_intron.copy()

In [4]:
# Parse out the name of the Gene from the gff file, by parsing the name column 

nombres = []
for i in df_intron['name']:
    i = i.split()[6]
    i = i.split("\"")[1]
    nombres.append(i)
serial = pd.Series(nombres)


df_intron['GeneID'] = nombres

df_intron['stuff'] = df_intron['name']
df_intron.head(1)

,Chr,notes,part,Start,End,dot1,Strand,dot2,name,GeneID,stuff
0,I,dexseq_prepare_annotation.py,intronic_part,4359,5194,.,-,.,"transcripts ""NA""; exonic_part_number "" 1""; ge...",WBGene00022277,"transcripts ""NA""; exonic_part_number "" 1""; ge..."


In [5]:
# create a number barcode for each gene; deseq2 doesnt like it when there are genes with the same name;
# so we neeed to give each intron its own unique number, the introns can then be merged together

Barcoded_Geneid = []
for index, row in df_intron.iterrows():
    a = row.stuff
    a = a.split()[4]
    a = a.split()[0]
    z = row.GeneID
    z += "_"+a
    z = z.split(";")[0]
    z = z.split("\"")[0]
    Barcoded_Geneid.append(z)
    
df_intron['Barcoded_GeneID'] = Barcoded_Geneid
df_intron.head(1)

,Chr,notes,part,Start,End,dot1,Strand,dot2,name,GeneID,stuff,Barcoded_GeneID
0,I,dexseq_prepare_annotation.py,intronic_part,4359,5194,.,-,.,"transcripts ""NA""; exonic_part_number "" 1""; ge...",WBGene00022277,"transcripts ""NA""; exonic_part_number "" 1""; ge...",WBGene00022277_1


In [8]:
# Make an SAF file that can be used by feature counts 

intron_SAF_barcode = df_intron[['Barcoded_GeneID', 'Chr', 'Start', 'End', 'Strand']]

intron_SAF_barcode = intron_SAF_barcode.rename(columns={'Barcoded_Geneid':'GeneID'})

intron_SAF_barcode.head(1)  # looks good

In [13]:
# Save as saf for feature counts

intron_SAF_barcode.to_csv('Barcoded_intron_saf.saf', sep='\t', index=None, quoting=csv.QUOTE_NONE)